In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline

In [ ]:
do = pd.read_csv('../input/housesalesprediction/kc_house_data.csv')

In [ ]:
do.head()

In [ ]:
oi = do.copy()#Pour utilizer ce dataframe dans la partie zipcodes
do[ ['sqft_lot','sqft_living','floors','sqft_above','sqft_basement','price','sqft_lot15','sqft_living15','bathrooms','bedrooms','grade'] ].describe().astype(int)

In [ ]:
do['price'].hist(bins=64)

In [ ]:
do.loc[do['sqft_living15']<250000 ,'sqft_living15'].hist(bins=100)

In [ ]:
do = do[ ['sqft_lot','sqft_living','floors','sqft_above','sqft_basement','price','sqft_lot15','sqft_living15','bathrooms','bedrooms','grade'] ]

In [ ]:
do['log_sqft_lot']=np.log(1+do['sqft_lot'])
do['log_sqft_living']=np.log(1+do['sqft_living'])
do['log_sqft_living15']=np.log(1+do['sqft_living15'])
do['log_sqft_lot15']=np.log(1+do['sqft_lot15'])
do['log_sqft_basement']=np.log(1+do['sqft_basement'])
do['log_sqft_above']=np.log(1+do['sqft_above'])
do['log_bedrooms']=np.log(1+do['bedrooms'])
do['log_price']=np.log(1+do['price'])
do[['log_sqft_lot','log_sqft_living','floors','log_price','log_sqft_living15','log_sqft_lot15','log_sqft_basement','log_sqft_above']].describe().astype(int)

In [ ]:
trn_do = do[0:17000] #80% des données apprentissage
test_do  = do[17000:] #20% des données pour le test

In [ ]:
do['log_sqft_living'].hist(bins=100)

In [ ]:
do[ ['sqft_lot','sqft_living','floors','sqft_above','sqft_basement','price','sqft_lot15','sqft_living15','bathrooms','bedrooms','grade'] ].describe().astype(int)

In [ ]:
trn_N = trn_do[ ['log_sqft_lot','log_sqft_living','floors','log_sqft_above','log_sqft_basement','log_sqft_lot15','log_sqft_living15','bathrooms','log_bedrooms','grade'] ].values
trn_M = trn_do[ 'log_price' ].values

In [ ]:
test_N = test_do[ ['log_sqft_lot','log_sqft_living','floors','log_sqft_above','log_sqft_basement','log_sqft_lot15','log_sqft_living15','bathrooms','log_bedrooms','grade'] ].values
test_M = test_do[ 'log_price' ].values

In [ ]:
print( trn_N.shape, test_N.shape )
print( trn_M.shape, test_M.shape )

In [ ]:
import tensorflow as tf
import keras
from keras import layers
from keras.models import Model
from tensorflow.keras.optimizers import SGD

In [ ]:
input1 = layers.Input( shape=(10,) ,name='entree' )
n    =  layers.Dense(500) ( input1 )
activ =  layers.Activation('relu') (n)
output = layers.Dense(1) (activ)

In [ ]:
model1 = Model( input1, output )

model1.summary()

In [ ]:
oupt = SGD() 

model1.compile( optimizer=oupt, loss='mse' )

In [ ]:
model1.fit( trn_N, trn_M, epochs=10 )

In [ ]:
do['price']

In [ ]:
predc = model1.predict(test_N)
predc
np.exp(predc)-1

In [ ]:
import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler 
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import StandardScaler

In [ ]:
do_maxmin = do[ ['sqft_lot','sqft_living','floors','sqft_above','sqft_basement','sqft_lot15','sqft_living15','bathrooms','bedrooms','grade'] ]

In [ ]:
scaler = MinMaxScaler ()

In [ ]:
scaler.fit(do_maxmin)

In [ ]:
N_scaled = scaler.transform (do_maxmin)

In [ ]:
N_scaled.shape

In [ ]:
M_scaler = MinMaxScaler ()

In [ ]:
price1 = do[['price']]

In [ ]:
M_scaler.fit(price1)

In [ ]:
M_scaled = M_scaler.transform (price1)

In [ ]:
M_scaled

In [ ]:
input2 = layers.Input( shape=(10,) ,name='entree' )
n    =  layers.Dense(700) ( input2 )
activ =  layers.Activation('relu') (n)
output = layers.Dense(1) (activ)

In [ ]:
modelmaxmin = Model( input2, output )

In [ ]:
modelmaxmin.summary()

In [ ]:
optminmax = SGD() 

In [ ]:
modelmaxmin.compile( optimizer=optminmax, loss='mse' )

In [ ]:
modelmaxmin.fit( N_scaled[0:17000], M_scaled[0:17000], epochs=10 )

In [ ]:
M_scaled[17000:]

In [ ]:
predct = modelmaxmin.predict(N_scaled[17000:])
predct = M_scaler.inverse_transform(predct)
predct

In [ ]:
do_Z_Score = do[ ['sqft_lot','sqft_living','floors','sqft_above','sqft_basement','sqft_lot15','sqft_living15','bathrooms','bedrooms','grade'] ]

In [ ]:
Z_Score_scaler = StandardScaler()

In [ ]:
Z_scaled = Z_Score_scaler.fit_transform(do_Z_Score)

In [ ]:
Z_scaled

In [ ]:
Z_score_price = do[['price']]

In [ ]:
Z_price_scaler = StandardScaler()
Z_priced = Z_price_scaler.fit_transform(Z_score_price)
Z_priced

In [ ]:
input3 = layers.Input( shape=(10,) ,name='entree' )
n    =  layers.Dense(700) ( input3 )
activ =  layers.Activation('relu') (n)
output = layers.Dense(1) (activ)

In [ ]:
model_Z_Score = Model( input3, output )

In [ ]:
model_Z_Score.summary()

In [ ]:
opt_Zscore = SGD() 

model_Z_Score.compile( optimizer=opt_Zscore, loss='mse' )

model_Z_Score.fit( Z_scaled[0:17000], Z_priced[0:17000], epochs=10 )

In [ ]:
price1[:6]

In [ ]:
predctZ = model_Z_Score.predict(Z_scaled[17000:])
predctZ = Z_price_scaler.inverse_transform(predctZ)
predctZ[:6]

In [ ]:
oi[['zipcode']]

In [ ]:
zip_dms = pd.get_dummies(oi['zipcode'])
do_Zipcodes = pd.DataFrame(N_scaled)

In [ ]:
zip_do = pd.concat([ do_Zipcodes, zip_dms],axis=1)
zip_do

In [ ]:
input4 = layers.Input( shape=(80,) ,name='entree' )
n    =  layers.Dense(500) ( input4)
activ =  layers.Activation('relu') (n)
output = layers.Dense(1) (activ)

In [ ]:
modelzipcodes = Model( input4, output )

In [ ]:
modelzipcodes.summary()

In [ ]:
opt_zipcodes = SGD() 
modelzipcodes.compile( optimizer=opt_zipcodes, loss='mse' )
modelzipcodes.fit( zip_do[0:17000], M_scaled[0:17000], epochs=10 )

In [ ]:
predct_Zipcodes = modelzipcodes.predict(zip_do[17000:])
predctEndMaxMin = M_scaler.inverse_transform(predct_Zipcodes)
predctEndMaxMin

In [ ]:
do_Zipcodes_Zscore = pd.DataFrame(Z_scaled)
zip_do_Zscore = pd.concat([ do_Zipcodes_Zscore, zip_dms],axis=1)
zip_do_Zscore

In [ ]:
input5 = layers.Input( shape=(80,) ,name='entree' )
n    =  layers.Dense(500) ( input5 )
activ =  layers.Activation('relu') (n)
output = layers.Dense(1) (activ)

In [ ]:
modelzipcodes_Zscore = Model( input5, output )

In [ ]:
modelzipcodes_Zscore.summary()

In [ ]:
opt_zipcodes_Zscore = SGD() 

modelzipcodes_Zscore.compile( optimizer=opt_zipcodes_Zscore, loss='mse' )

modelzipcodes_Zscore.fit( zip_do_Zscore[0:17000], Z_priced[0:17000], epochs=10 )

In [ ]:
predct_Zipcodes_Zscore = modelzipcodes_Zscore.predict(zip_do_Zscore[17000:])
predct_End_Zscore = Z_price_scaler.inverse_transform(predct_Zipcodes_Zscore)
predct_End_Zscore